In [1]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
from skimage import measure
import csv
from collections import defaultdict

## Preprocess Image
---
1. Loads the image directly in grayscale. It is a simple and fast conversion
2. Enhances the local contrast in the image:
> Using a medium-size tileGridSize for a contrast that is neither too local nor too global.
> Using a medium-size clipLimit to increase the contrast, but it is not a high value so as not to cause excessive noise, but keep in mind that some people can be seen incompletely or far away, so a low value does not work.

3. Apply Gaussian: We want to reduce noise, but since the people are small silhouettes, we don't want their details to disappear, so we set the kernel smaller. The standard deviation is set to 0 for automatic adjustment. This setting consists of OpenCV calculating a value based on the kernel size, balancing smoothing and detail preservation.
4. Maintain detail in darker areas while controlling the effect of the overexposed regions. thresh 127 is balanced, if it is smaller it causes overexposure, and if it is larger it causes more “noise”.

In [2]:
def load_image(image_name, show_image=False):
    image_path=f"data/{image_name}"
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    if image is None:
        print(f"The image cannot be loaded: {image_path}")
        return None
    if show_image:
        plt.imshow(image_path)
    return image

def clahe(image: np.ndarray,show_image=False, clip_limit=20.0, grid_size=(14, 14)) -> np.ndarray:
    """Contrast-limited adaptive histogram equalization."""
    image = image.astype('uint8')   # Ensure that the image is of type uint8
    clahe = cv2.createCLAHE(clipLimit=clip_limit, tileGridSize=grid_size)
    if show_image:
        plt.title('Clahe')
        plt.imshow(clahe)
    return clahe.apply(image)

def preprocess_image(image_name,show_image=False):
    image_grayscale = load_image(image_name,show_image)
    if image_grayscale is None:
        return None

    img_equalized = clahe(image_grayscale,show_image)
    blurred_image = cv2.GaussianBlur(img_equalized, (3, 3), 0)
    _, binary_image = cv2.threshold(blurred_image, 127, 255, cv2.THRESH_TRUNC)
    return binary_image

## Extract differences
---
1. The XOR operator will highlight the differences between the two images, which in this case will be the additional objects on the background
2. Apply opening to remove small noise

In [3]:
def segmentation_foreground(binary_image: np.ndarray, binary_bckg_image: np.ndarray,show_image=False):
    segmented_image = cv2.bitwise_xor(binary_bckg_image, binary_image)
    # Delete small differences
    kernel = np.ones((5, 5), np.uint8)
    refined_image = cv2.morphologyEx(segmented_image, cv2.MORPH_OPEN, kernel)
    return refined_image

## Detect edges
---
1. Detect edges and using "adhoc" numbers
2. Enhance edges improve to separate them from the background

In [4]:
def edge_detection(image: np.ndarray,show_image=False, min_threshold=230, max_threshold=180):
    edges = cv2.Canny(image, min_threshold, max_threshold)
    intensified_edges = cv2.Laplacian(edges, cv2.CV_64F)
    return intensified_edges

## Obtain person detected coordinates
---
1. Convert image from float64 to 8uint and 1 channel to be used by findContours
2. Obtain contours, and filter by the perimeter size
3. Obtain the center points of the contours

In [5]:
def obtain_contours(image: np.ndarray,show_image=False):
    image_8b_c1 = cv2.convertScaleAbs(image, alpha=(255.0/np.max(image)))
    contours, _ = cv2.findContours(image_8b_c1, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    min_perimeter = 50
    filtered_contours = [cnt for cnt in contours if cv2.arcLength(cnt, True) > min_perimeter]
    return filtered_contours
def obtain_center_points(filtered_contours,show_image=False):
    y_min = 420
    contour_points = []
    for contorno in filtered_contours:
        M = cv2.moments(contorno)
        if M['m00'] != 0:
            cx = int(M['m10'] / M['m00'])
            cy = int(M['m01'] / M['m00'])
            if cy > y_min:
                contour_points.append((cx, cy))
    return contour_points

In [6]:
def process_image(image_path, bckg_image,show_image=False):
    binary_image = preprocess_image(image_path)
    if binary_image is None:
        return None

    segmented_image = segmentation_foreground(binary_image, bckg_image,show_image)
    edges = edge_detection(segmented_image,show_image)
    contours = obtain_contours(edges,show_image)
    center_points = obtain_center_points(contours,show_image)
    return {
        'image': image_path,
        'count': len(center_points),
        'points': center_points
    }

In [7]:
def process_images(image_names, bckg_image,show_image=False):
    processed_images = []
    for name in image_names:
        processed_image = process_image(name, bckg_image,show_image)
        if processed_image is not None:
            processed_images.append(processed_image)
    return processed_images

In [8]:
def read_manual_annotations(input_file,with_headers=False):
    file_path=f"data/{input_file}"
    # Diccionario para almacenar los datos
    data = defaultdict(lambda: {'people_count': 0, 'coordinates': []})
    # No headers
    with open(file_path, 'r', newline='') as csvfile:
        reader = csv.reader(csvfile)
        for row in reader:
            if with_headers:
                name = row['name']
                x = int(row['x'])
                y = int(row['y'])
            else:  
                # Extract values based on the order: label, x, y, name, height, width
                name = row[3]
                x = int(row[1])
                y = int(row[2])
            data[name]['people_count'] += 1
            data[name]['coordinates'].append((x, y))
    return data

In [9]:
def validation(manual_results, results):
    comparison = []
    for result in results:
        image_name = result['image']
        manual_data = manual_results.get(image_name, {'people_count': 0, 'coordinates': []})
        manual_count = manual_data['people_count']
        manual_points = manual_data['coordinates']
        detected_count = result['count']
        detected_points = result['points']
        #Has to imrpove to set a proper distance, and cases where multiple prediction dots are on the same person
        close_points = sum(1 for dp in detected_points if any(np.linalg.norm(np.array(dp) - np.array(mp)) < 20 for mp in manual_points))
        comparison.append({
            'image': image_name,
            'manual_count': manual_count,
            'detected_count': detected_count,
            'close_points': close_points
        })
    return comparison

### Validation
---
**Image level**

In [10]:
manual_path = "manual_annotations.csv"
manual_results = read_manual_annotations(manual_path)
bckg_image_name = '1660284000.jpg'
bckg_image = preprocess_image(bckg_image_name)
image_names = ["1660287600.jpg", "1660294800.jpg","1660320000.jpg"]
results = process_images(image_names, bckg_image)
comparison = validation(manual_results, results)

for comp in comparison:
    mse = np.mean((comp['manual_count'] - comp['detected_count']) ** 2) #sustraendo debe ser la predicción
    print(f"Image: {comp['image']}, Manual Count: {comp['manual_count']}, Detected Count: {comp['detected_count']}, MSE Image level: {mse}, Close Points: {comp['close_points']}")

Image: 1660287600.jpg, Manual Count: 19, Detected Count: 107, MSE Image level: 7744.0, Close Points: 8
Image: 1660294800.jpg, Manual Count: 66, Detected Count: 139, MSE Image level: 5329.0, Close Points: 16
Image: 1660320000.jpg, Manual Count: 155, Detected Count: 218, MSE Image level: 3969.0, Close Points: 66


Notes:  
- Currently, the background image keeps with the 2 persons.
- They are not equally enlightened in the same way; however, on the test done, the contrast is lowered (in the same image).
- Not establishing specific contours for a person (different cases).
. How do you achieve accuracy with the points? Establish a certain distance.

